# Basketball Lineup Optimization

## Problem

Given a basketball roster with 13 players who each have a constrained number of minutes they can play in a game and a plus/minus impact value for when they are in, assign the 200 playable minutes in a way that maximizes the total plus/minus for the team.

## Methods for Optimizing

#### The methods for constrained optimization that I found are as follows:

#### 1. Penalty Method

This method enforces the minutes restrictions by assigning a penalty for breaking them. This is not ideal for this problem because the constraints cannot be violated at all.

#### 2. Lagrange Multipliers

This method solves the problem using a system of equations based on the conditions. This is impractical for this problem because there would be far too many equations to solve.

#### 3. Interior Point Method

Uses barrier functions that will blow up to infinity when the constraints are hit. This is not good for this problem though because the constraint on total minutes played is an equality, not a "barrier".

#### 4. Augmented Lagrangian Method

This combination of the penalty method and lagrangian methods is not good either since the two methods it combines do not apply well to this problem.

#### 5. Sequential Quadratic Programming

Approximate the problem at each iteration as a quadratic program (similar to Newton's Method). This method has no obvious downside for this problem so it is possible to look at an example:

In [14]:
# Import packages
import numpy as np
from scipy.optimize import minimize

# Define the number of players and the total minutes for the game
num_players = 6
total_minutes = 200.0

# Assign each players minutes boundaries
bounds = [
    (0, 40),   # x[0]
    (0, 40),   # x[1]
    (0, 40),   # x[2]
    (0, 40),   # x[3]
    (0, 40),   # x[4]
    (0, 40),   # x[5]
]

# Enter each player plus/minus per minute they are on the court
plusminuses = [2.5, 1.4, 0, -1.1, -2.4, -3]

# Set the function that will be maximized to equal the sum of every players plus/minus per minute times their minutes played
def score(x):
    return sum(x[i] * plusminuses[i] for i in range(num_players))

# Set the objective to be maximizing this output (minizing the inverse)
def objective(x):
    return -score(x)

# Force the total number of minutes to be equal to 200 (this is the step that several of the other methods can't handle)
constraints = {
    'type': 'eq',
    'fun': lambda x: np.sum(x) - total_minutes
}

# Initially assign all players the same number of minutes
x0 = np.full(num_players, total_minutes / num_players)

# Solve the problem using the Sequential Least Squares Quadratic Program, which will take a step in the direction of
# the solution using the partial derivatives, similar to Newton's Method
result = minimize(
    fun=objective,
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

# Print the output
print("Optimal Rotation:")
for i in range(num_players):
    print(f"  Player {i}: {result.x[i]:5.2f} minutes (plus/minus: {plusminuses[i]:+2.2f})")
print(f"\nTotal minutes: {np.sum(result.x):.2f}")
print(f"Team +/-: {-result.fun:.2f}")
print(f"Initial plus/minus: {score(x0):.2f}") # could be replaced by the real game plus/minus for the practical application
print(f"Improvement: {-result.fun - score(x0):.2f}")

Optimal Rotation:
  Player 0: 40.00 minutes (plus/minus: +2.50)
  Player 1: 40.00 minutes (plus/minus: +1.40)
  Player 2: 40.00 minutes (plus/minus: +0.00)
  Player 3: 40.00 minutes (plus/minus: -1.10)
  Player 4: 40.00 minutes (plus/minus: -2.40)
  Player 5:  0.00 minutes (plus/minus: -3.00)

Total minutes: 200.00
Team +/-: 16.00
Initial plus/minus: -86.67
Improvement: 102.67


This solver provides the correct solution for the simplified, example problem I provided.

I envision that we could build upon this by expanding it to a full roster of players, adding their real plus/minus per minute from last year and then adding a way to calculate the real team plus/minus for the game and check whether the solver' suggestions would be an improvement and measure how much.

If this all goes well we could take it a step further by adding player interaction metrics to help account for the impact players make on their teammates instead of simply summing individual player plus minus to assign lineups plus/minus for example. If this makes an impractical number of possible lineups then we could choose a different route for the advancement. 